In [1]:
import numpy as np
import pythreejs
from moon import io as mio

range_x = np.array([42400, 43850])
range_y = np.array([33500, 34750])
shape_large = np.array([46080, 92160])
range_lon = (range_x / shape_large[1] * 360 - 180)
range_lat = (90 - range_y / shape_large[0] * 180)

# FIXME!!!! We need to pass it a square? Come on!
def central_square(arr, lats, lons):
    """Trims an array to a largest central square"""
    
    crop = min(arr.shape)
    startx = arr.shape[1]//2-(crop//2)
    starty = arr.shape[0]//2-(crop//2)
    
    return (arr[starty:starty+crop, startx:startx+crop],
            lats[starty:starty+crop], lons[startx:startx+crop])

# pythreejs can't handle > 2 MB arrays?
n_drop = 5
x = np.linspace(*range_lon, int(np.diff(range_x)))[::n_drop]
y = np.linspace(*range_lat, int(np.diff(range_y)))[::n_drop]
z = mio.get_tycho_cutout()[::n_drop, ::n_drop].astype(np.float32) / 1e4
z, x, y = central_square(z, y, x)

ny, nx = z.shape

In [2]:
surf_g = pythreejs.SurfaceGeometry(z=list(z.flat), width=x.max()-x.min(), height=y.max()-y.min(),
                                   width_segments=nx-1, height_segments=ny-1)

surf = pythreejs.Mesh(geometry=surf_g, 
                      material=pythreejs.MeshLambertMaterial(map=pythreejs.height_texture(z, 'YlGnBu_r')))
surfgrid = pythreejs.SurfaceGrid(geometry=surf_g,
                                 material=pythreejs.LineBasicMaterial(color='black'))
scene = pythreejs.Scene(fog=pythreejs.Fog(near=2,far=7),
                        children=[surf, surfgrid,
                        pythreejs.AmbientLight(color='#777777')])
dlight = pythreejs.DirectionalLight(color='white', position=[3, 5, 1],
                                    intensity=0.6)
c = pythreejs.PerspectiveCamera(position=[0, 3, 3], up=[0, 0, 1], 
                                children=[dlight])
renderer = pythreejs.Renderer(camera=c, scene=scene,
                              width=900, height=600,
                              controls=[pythreejs.OrbitControls(controlling=c)])
renderer

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.0, 5.0,…

### TODOs:

1. Make a PR for broken numpy in SurfaceGeometry update method?
2. Fix the annoying non-square restriction - in particular, the texture doesn't work at all? 
3. Get the resterio geoslicing working and allow switching between crater names